In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import importlib as imp
import traditional
# models to use
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
# from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

In [3]:
imp.reload(traditional)
r = 0 # global random state
number_of_folds = 5 # is actually 5
raw = pd.read_csv('input_data_X_531_demo_1.csv', index_col=None)
# naively fill nan with 0
raw = raw.fillna(value=0)
# naively replace infinite values with 0
raw = raw.replace([np.inf, -np.inf], 0)
# the first column is read as 'Unnamed: 0'
raw.rename(columns={'Unnamed: 0': "SUBJECT_ID"}, inplace=True)
raw.head()

,SUBJECT_ID,220180__maximum,220180__minimum,220180__mean,220180__kurtosis,220180__skewness,220180__standard_deviation,220180__root_mean_square,220180__sample_entropy,220210__maximum,...,220051__root_mean_square,220051__sample_entropy,646__maximum,646__minimum,646__mean,646__kurtosis,646__skewness,646__standard_deviation,646__root_mean_square,646__sample_entropy
0,23,75.0,45.0,62.187500,0.567319,-0.781028,7.779932,62.672263,0.000000,23.0,...,68.409236,1.386294,100.0,93.0,97.906977,-1.195245,-0.433719,2.122021,97.929970,0.228439
1,34,100.0,33.0,59.888889,1.558259,0.822361,12.937409,61.270348,0.000000,26.0,...,0.000000,0.000000,97.0,89.0,93.464286,-0.939317,-0.351776,2.162706,93.489304,1.609438
2,36,100.0,48.0,71.390000,-0.433960,0.219646,11.788889,72.356824,1.945910,28.0,...,0.000000,0.000000,100.0,0.0,94.765217,38.053458,-5.818812,12.918696,95.641723,0.383990
3,85,101.0,42.0,63.687500,-0.492475,0.719208,15.219445,65.480754,1.421386,39.0,...,0.000000,0.000000,100.0,94.0,97.734177,-1.079023,-0.304174,1.791039,97.750587,1.119666
4,107,115.0,11.0,50.697368,0.156891,0.108199,21.412625,55.033841,0.988611,29.0,...,0.000000,0.000000,100.0,95.0,96.875000,0.666985,0.666902,1.268611,96.883306,0.693147


In [4]:
admissions = pd.read_csv('mimic-iii-clinical-database-1.4/ADMISSIONS.csv')
admissions.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [13]:
raw['SUBJECT_ID']

0          23
1          34
2          36
3          85
4         107
        ...  
4995    51550
4996    51551
4997    51555
4998    51558
4999    51582
Name: SUBJECT_ID, Length: 5000, dtype: int64

In [37]:
"""Use SUBJECT_ID in 'raw' to select icustays entries.
Then, select the first stay duration for each SUBJECT_ID"""
icustays = pd.read_csv('mimic-iii-clinical-database-1.4/ICUSTAYS.csv')
icustays = icustays[['SUBJECT_ID', 'LOS']]
subset = icustays[icustays['SUBJECT_ID'].isin(raw['SUBJECT_ID'])]
subset = subset.drop_duplicates(subset='SUBJECT_ID', keep="first")
subset.head()

,SUBJECT_ID,LOS
24,291,0.5102
41,305,3.9185
129,85,2.2353
155,107,1.0186
159,109,0.8935


In [38]:
filtered_admissions = admissions.merge(raw, how='inner')
filtered_admissions = filtered_admissions.merge(icustays, how='outer')
# a patient gets admitted multiple times; I keep the last instance
filtered_admissions = filtered_admissions.drop_duplicates(subset='SUBJECT_ID', keep="last")
filtered_admissions.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,...,220051__sample_entropy,646__maximum,646__minimum,646__mean,646__kurtosis,646__skewness,646__standard_deviation,646__root_mean_square,646__sample_entropy,LOS
3,23.0,23,124321.0,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,...,1.386294,100.0,93.0,97.906977,-1.195245,-0.433719,2.122021,97.929970,0.228439,1.1862
7,34.0,34,144319.0,2191-02-23 05:23:00,2191-02-25 20:20:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,...,0.000000,97.0,89.0,93.464286,-0.939317,-0.351776,2.162706,93.489304,1.609438,1.5824
16,38.0,36,165660.0,2134-05-10 11:30:00,2134-05-20 13:16:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,LONG TERM CARE HOSPITAL,Medicare,...,0.000000,100.0,0.0,94.765217,38.053458,-5.818812,12.918696,95.641723,0.383990,4.3483
41,460.0,357,117876.0,2199-12-21 22:37:00,2200-01-18 11:45:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,...,0.000000,100.0,87.0,98.239416,1.970949,-1.132976,1.665772,98.253538,0.636562,14.0641
45,88.0,85,112077.0,2167-07-25 18:49:00,2167-07-30 15:24:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,Medicare,...,0.000000,100.0,94.0,97.734177,-1.079023,-0.304174,1.791039,97.750587,1.119666,1.9909


In [69]:
imp.reload(traditional)
r = 0 # global random state
number_of_folds = 5 # is actually 5
raw = pd.read_csv('input_data_X_531_demo_1.csv', index_col=None)
# statistical features Daoyi extracted with tsfresh
admissions = pd.read_csv('mimic-iii-clinical-database-1.4/ADMISSIONS.csv')
# naively fill nan with 0
raw = raw.fillna(value=0)
# naively replace infinite values with 0
raw = raw.replace([np.inf, -np.inf], 0)
# the first column is read as 'Unnamed: 0'
raw.rename(columns={'Unnamed: 0': "SUBJECT_ID"}, inplace=True)

In [71]:
"""Labels: mortality,
TODO: labels for LOS column"""
labels = np.asarray(filtered_admissions['HOSPITAL_EXPIRE_FLAG'])
labels.shape

(5000,)

In [72]:
"""Inputs: all colums except the first (SUBJECT_ID)"""
inputs = np.asarray(raw[raw.columns[1:]])
inputs.shape

(5000, 64)

In [91]:
from sklearn.model_selection import KFold, StratifiedKFold
x_cv, x_test, y_cv, y_test = sk.model_selection.train_test_split(
    inputs, labels, test_size=0.2, random_state=r)
kf = StratifiedKFold(n_splits=number_of_folds, shuffle=False)
train_indices = []
valid_indices = []
for i, (train_index, valid_index) in enumerate(kf.split(x_cv, y_cv)):
    train_indices.append(train_index)
    valid_indices.append(valid_index)
train_indices = np.asarray(train_indices)
valid_indices = np.asarray(valid_indices)

In [92]:
imp.reload(traditional)
from sklearn.metrics import roc_auc_score, average_precision_score
for model in [
    SVC(random_state=r, probability=True),
    LogisticRegression(random_state=r),
    RandomForestClassifier(random_state=r),
    GaussianNB(),
    XGBClassifier(random_state=r)
]:
    print(f"<Model>: {model}")
    classifier = traditional.cross_validation(
        sklearn_classifier=model,
        x_cv=x_cv, y_cv=y_cv, train_indices=train_indices, valid_indices=valid_indices)
    scores_test = classifier.predict_proba(x_test)[:, 1]
    AUROC = roc_auc_score(y_true=y_test, y_score=scores_test)
    AUPRC = average_precision_score(y_true=y_test, y_score=scores_test)
    print(f"TESTING",
        f"\n\tAUROC: {AUROC}",
        f"\n\tAUPRC: {AUPRC}")

<Model>: SVC(probability=True, random_state=0)
	fold 1: 	fold 2: 	fold 3: 	fold 4: 	fold 5: 
TRAIN-AND-VALIDATE 
	AUROC:      mean 0.5221 
	            std  0.0095 
	AUPRC:      mean 0.1556 
	            std  0.0095
Best classifier is in fold: 4
TESTING 
	AUROC: 0.48320340184266475 
	AUPRC: 0.1623265995935155
<Model>: LogisticRegression(random_state=0)
	fold 1: 	fold 2: 	fold 3: 	fold 4: 	fold 5: 
TRAIN-AND-VALIDATE 
	AUROC:      mean 0.5032 
	            std  0.0080 
	AUPRC:      mean 0.1535 
	            std  0.0080
Best classifier is in fold: 2
TESTING 
	AUROC: 0.5116087880935506 
	AUPRC: 0.17179418685720393
<Model>: RandomForestClassifier(random_state=0)
	fold 1: 	fold 2: 	fold 3: 	fold 4: 	fold 5: 
TRAIN-AND-VALIDATE 
	AUROC:      mean 0.5082 
	            std  0.0119 
	AUPRC:      mean 0.1481 
	            std  0.0119
Best classifier is in fold: 0
TESTING 
	AUROC: 0.5055386250885896 
	AUPRC: 0.16737889659497063
<Model>: GaussianNB()
	fold 1: 	fold 2: 	fold 3: 	fold 4: 	fold 5: 
T